In [ ]:
import sys
MAINPATH = ".."  # nopep8
sys.path.append(MAINPATH)  # nopep8
from src.models.model import NBeatsNet
import numpy as np

In [2]:
num_samples, time_steps, input_dim, output_dim = 50_000, 10, 1, 1

In [3]:
model = NBeatsNet(
        backcast_length=time_steps, forecast_length=output_dim,
        nb_blocks_per_stack=2, thetas_dim=(4, 4), share_weights_in_stack=True,
        hidden_layer_units=64
    )

| N-Beats
| --  Stack Trend (#0) (share_weights_in_stack=True)
     | -- TrendBlock(units=64, thetas_dim=4, backcast_length=10, forecast_length=1, share_thetas=True) at @5898132736
     | -- TrendBlock(units=64, thetas_dim=4, backcast_length=10, forecast_length=1, share_thetas=True) at @5898132736
| --  Stack Seasonality (#1) (share_weights_in_stack=True)
     | -- SeasonalityBlock(units=64, thetas_dim=1, backcast_length=10, forecast_length=1, share_thetas=True) at @5898133072
     | -- SeasonalityBlock(units=64, thetas_dim=1, backcast_length=10, forecast_length=1, share_thetas=True) at @5898133072


In [4]:
# Definition of the objective function and the optimizer.
model.compile(loss='mae', optimizer='adam')

In [5]:
# Definition of the data. The problem to solve is to find f such as | f(x) - y | -> 0.
# where f = np.mean.
x = np.random.uniform(size=(num_samples, time_steps, input_dim))
y = np.mean(x, axis=1, keepdims=True)

# Split data into training and testing datasets.
c = num_samples // 10
x_train, y_train, x_test, y_test = x[c:], y[c:], x[:c], y[:c]
test_size = len(x_test)


In [12]:
x.shape, y.shape

((50000, 10, 1), (50000, 1, 1))

In [5]:
# Train the model.
print('Training...')
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=128)

# Save the model for later.
#model.save('n_beats_model.h5')

# Predict on the testing set (forecast).
predictions_forecast = model.predict(x_test)
np.testing.assert_equal(predictions_forecast.shape, (test_size, model.forecast_length, output_dim))

# Predict on the testing set (backcast).
predictions_backcast = model.predict(x_test, return_backcast=True)
np.testing.assert_equal(predictions_backcast.shape, (test_size, model.backcast_length, output_dim))



Training...
Epoch 01/20
352/352 [==============================] - 0s 1ms/step - loss: 0.1131 - val_loss: 0.0088
Epoch 02/20
352/352 [==============================] - 0s 1ms/step - loss: 0.0046 - val_loss: 0.0030
Epoch 03/20
352/352 [==============================] - 0s 1ms/step - loss: 0.0026 - val_loss: 0.0021
Epoch 04/20
352/352 [==============================] - 0s 1ms/step - loss: 0.0020 - val_loss: 0.0017
Epoch 05/20
352/352 [==============================] - 0s 1ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 06/20
352/352 [==============================] - 0s 1ms/step - loss: 0.0014 - val_loss: 0.0017
Epoch 07/20
352/352 [==============================] - 0s 1ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 08/20
352/352 [==============================] - 0s 1ms/step - loss: 0.0011 - val_loss: 0.0009
Epoch 09/20
352/352 [==============================] - 0s 1ms/step - loss: 0.0010 - val_loss: 0.0009
Epoch 10/20
352/352 [==============================] - 0s 1ms/step - loss: 0.00